In [2]:
from transformers import * 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import random 
import re 
import math 
from tqdm import tqdm 
import sklearn 
import torch 
import torch.nn as nn 
import torch.functional as f
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from sklearn.model_selection import train_test_split 
import time 
import datetime


In [3]:
train = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv") 

train.shape, test.shape, train['label'].nunique()


((174304, 13), (43576, 12), 46)

In [4]:
zero_train = train[train['label'] == 0] 
nonzero_train = train[train['label'] != 0]

In [5]:
zero_train.shape, nonzero_train.shape


((142571, 13), (31733, 13))

In [6]:
# convert all nonzero_train data sample's labels to 1 
nonzero_train['label'] = 1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
train_df = pd.concat([zero_train, nonzero_train], axis = 0)

In [8]:
def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean


In [9]:
def split_text(s, overlap = 20, chunk_size = 50): 
    total = [] 
    partial = [] 
    if len(s.split()) // (chunk_size - overlap) > 0:  
        n = len(s.split()) // (chunk_size - overlap) 
    else: 
        n = 1 
    for w in range(n): 
        if w == 0: 
            partial = s.split()[:chunk_size] 
            total.append(" ".join(partial)) 
        else:  
            partial = s.split()[w*(chunk_size - overlap):w*(chunk_size - overlap) + chunk_size]
            total.append(" ".join(partial)) 
    return total


In [10]:
train_df['요약문_내용'] = train_df['요약문_연구목표'] + train_df['요약문_연구내용'] + train_df['요약문_기대효과'] 
test['요약문_내용'] = test['요약문_연구목표'] + test['요약문_연구내용'] + test['요약문_기대효과']


In [11]:
train_df['사업명'].fillna('NAN',inplace=True) 
train_df['사업_부처명'].fillna('NAN',inplace=True) 
train_df['내역사업명'].fillna('NAN',inplace=True) 
train_df['과제명'].fillna('NAN',inplace=True) 
train_df['요약문_한글키워드'].fillna('NAN',inplace=True)


In [12]:
train_df['요약문_내용'].fillna('NAN',inplace=True) 
test['요약문_내용'].fillna('NAN',inplace=True)


In [13]:
contents = train_df['요약문_내용'].values 
feature1 = train_df['사업명'].values 
feature2 = train_df['사업_부처명'].values 
feature3 = train_df['내역사업명'].values 
feature4 = train_df['과제명'].values 
feature5 = train_df['요약문_한글키워드'].values 
feature6 = train_df['label'].values 

train_data = {'사업명':[],'사업_부처명':[],'내역사업명':[],'과제명':[],'한글키워드':[],'요약문':[],'label':[]} 

for i in tqdm(range(contents.shape[0]), position = 0, leave = True): 
    sample = str(contents[i]) 
    splitted_text = split_text(clean_text(sample)) 
    for t in splitted_text: 
        train_data['요약문'].append(t) 
        train_data['사업명'].append(clean_text(str(feature1[i])))
        train_data['사업_부처명'].append(clean_text(str(feature2[i]))) 
        train_data['내역사업명'].append(clean_text(str(feature3[i]))) 
        train_data['과제명'].append(clean_text(str(feature4[i])))  
        train_data['한글키워드'].append(feature5[i]) # no cleaning for this one
        train_data['label'].append(feature6[i])
        
    

100%|██████████| 174304/174304 [01:47<00:00, 1622.88it/s]


In [14]:
train_data = pd.DataFrame(train_data)
train_data.head(5)

,사업명,사업_부처명,내역사업명,과제명,한글키워드,요약문,label
0,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",최종목표 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구 발굴된 유전자를 통한 ...,0
1,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",저항성 극복 기전을 규명 환자조직 동물실험 세포실험을 통해 대장암에 특이적으로 조절...,0
2,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",내성 유발 유전자를 발굴하고 저항성 극복 기전을 규명 추후 기반 항암화학요법 치료효...,0
3,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",대장암 환자조직을 이용하여 후보 유전자의 발현과 발현 양상 분석 후보 유전자 발현 ...,0
4,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",표적 가능성 검증 선천적 내성 예측인자 발굴 차년도 를 통한 후천적 내성 표적 후보...,0


In [15]:
## Now we tokenize each data and make sure they all lie within the 512 tokenization range 
## if not check how many have token length greater than 512 

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator") 

def electra_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences, not "really" necessary for now    
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:512] 
        attention_mask = attention_mask[:512] 
        token_type_id = token_type_id[:512]  
        print("Long Text!! Truncating to the first 512 tokens!")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        token_type_id = token_type_id + [0]*(512 - len(token_type_id))  
        
    return input_id, attention_mask, token_type_id


In [16]:
# define important hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 30
VALID_SPLIT = 0.1 
MAX_LEN = 512 # max token size for BERT, ELECTRA

In [17]:
train_data['data'] = train_data['사업명'] + " " + train_data['사업_부처명'] + " " + train_data['내역사업명'] + " " + train_data['과제명'] + " " + train_data['한글키워드'] + " " + train_data['요약문'] 

train_data.head(2)


,사업명,사업_부처명,내역사업명,과제명,한글키워드,요약문,label,data
0,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",최종목표 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구 발굴된 유전자를 통한 ...,0,이공학학술연구기반구축 교육부 지역대학우수과학자지원사업 년 년 대장암의...
1,이공학학술연구기반구축,교육부,지역대학우수과학자지원사업 년 년,대장암의 내성 표적 인자 발굴 및 반응 예측 유전자 지도 구축...,"대장암,항암제 내성,세포사멸,유전자발굴",저항성 극복 기전을 규명 환자조직 동물실험 세포실험을 통해 대장암에 특이적으로 조절...,0,이공학학술연구기반구축 교육부 지역대학우수과학자지원사업 년 년 대장암의...


In [18]:
train_text = train_data['data'].values 
train_labels = train_data['label'].values


In [19]:
N = train_data.shape[0] 

input_ids = np.zeros((N, MAX_LEN),dtype=int)
attention_masks = np.zeros((N, MAX_LEN),dtype=int)
token_type_ids = np.zeros((N, MAX_LEN),dtype=int) 
labels = np.zeros((N),dtype=int)

for i in tqdm(range(N), position=0, leave=True): 
    try:
        cur_str = train_text[i]
        cur_label = train_labels[i]
        input_id, attention_mask, token_type_id = electra_tokenizer(cur_str, MAX_LEN=MAX_LEN) 
        input_ids[i,] = input_id 
        attention_masks[i,] = attention_mask 
        token_type_ids[i,] = token_type_id
        labels[i] = cur_label 
    except Exception as e: 
        print(e)
        print(cur_str)
        pass


  4%|▍         | 65474/1638867 [02:33<1:21:21, 322.34it/s]

Long Text!! Truncating to the first 512 tokens!


  6%|▌         | 100841/1638867 [04:10<57:09, 448.49it/s]  

Long Text!! Truncating to the first 512 tokens!


  7%|▋         | 113210/1638867 [04:45<1:21:02, 313.79it/s]

...

 95%|█████████▌| 1558171/1638867 [1:03:32<04:05, 328.35it/s]

Long Text!! Truncating to the first 512 tokens!


 96%|█████████▌| 1569286/1638867 [1:04:03<02:51, 405.50it/s]

Long Text!! Truncating to the first 512 tokens!


100%|██████████| 1638867/1638867 [1:07:26<00:00, 405.01it/s]


In [20]:
input_ids = torch.tensor(input_ids, dtype=int)
attention_masks = torch.tensor(attention_masks, dtype=int)
token_type_ids = torch.tensor(token_type_ids, dtype=int) 
labels = torch.tensor(labels, dtype=int)


In [21]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, random_state = 42, test_size = VALID_SPLIT, stratify=labels) 

train_attention_mask, val_attention_mask, _, _ = train_test_split(attention_masks, labels, random_state = 42, test_size = VALID_SPLIT, stratify=labels) 

train_token_ids, val_token_ids, _, _ = train_test_split(token_type_ids, labels, random_state = 42, test_size = VALID_SPLIT, stratify=labels) 


train_inputs.shape, train_attention_mask.shape, train_token_ids.shape, train_labels.shape


(torch.Size([1474980, 512]),
 torch.Size([1474980, 512]),
 torch.Size([1474980, 512]),
 torch.Size([1474980]))

In [22]:
train_data = TensorDataset(train_inputs, train_attention_mask, train_token_ids, train_labels) 
train_sampler = RandomSampler(train_data) 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE) 

validation_data = TensorDataset(val_inputs, val_attention_mask, val_token_ids, val_labels) 
validation_sampler = SequentialSampler(validation_data) 
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

In [23]:
# check if label is zero
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=2)
model.cuda()


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [24]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

epochs = 10 

total_steps = len(train_dataloader) * epochs 

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

def flat_accuracy(preds, labels): 
    pred_flat = np.argmax(preds, axis=1).flatten() 
    labels_flat = labels.flatten() 
    return np.sum(pred_flat == labels_flat) / len(labels_flat) 

def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


device = torch.device("cuda")


# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 20 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print('  current average loss = {}'.format(total_loss / step))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=b_token_type_ids, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=b_token_type_ids, 
                            attention_mask=b_input_mask, 
                            labels = b_labels)
    
        
        loss = outputs[0] 
        logits = outputs[1] 
        
        # 로스 구함 
        eval_loss += loss.item() 
        
        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    
    avg_val_loss = eval_loss / len(validation_dataloader)            
    print("  Average validation loss: {}".format(avg_val_loss))
    print("  Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    
    torch.save(model.state_dict(), "ELECTRA_ZERO_MODEL_" + str(epoch_i + 1)) 
print("")
print("Training complete!")



======== Epoch 1 / 10 ========
Training...
  Batch    20  of  46,094.    Elapsed: 0:00:20.
  current average loss = 0.540230070054531
  Batch    40  of  46,094.    Elapsed: 0:00:39.
  current average loss = 0.502000055462122
  Batch    60  of  46,094.    Elapsed: 0:00:58.
  current average loss = 0.48806283821662266
  Batch    80  of  46,094.    Elapsed: 0:01:17.
  current average loss = 0.4912450142204762
  Batch   100  of  46,094.    Elapsed: 0:01:36.
  current average loss = 0.4852678745985031
  Batch   120  of  46,094.    Elapsed: 0:01:56.
  current average loss = 0.46346937827765944
  Batch   140  of  46,094.    Elapsed: 0:02:15.
  current average loss = 0.44974800891109873
  Batch   160  of  46,094.    Elapsed: 0:02:34.
  current average loss = 0.4368922790512443
  Batch   180  of  46,094.    Elapsed: 0:02:53.
  current average loss = 0.42465985963741937
  Batch   200  of  46,094.    Elapsed: 0:03:13.
  current average loss = 0.41486721552908423
  Batch   220  of  46,094.    Ela

  Batch 1,800  of  46,094.    Elapsed: 0:28:54.
  current average loss = 0.254959768967496
  Batch 1,820  of  46,094.    Elapsed: 0:29:13.
  current average loss = 0.2543089690075798
  Batch 1,840  of  46,094.    Elapsed: 0:29:32.
  current average loss = 0.2542325887748081
  Batch 1,860  of  46,094.    Elapsed: 0:29:51.
  current average loss = 0.25403799429055185
  Batch 1,880  of  46,094.    Elapsed: 0:30:11.
  current average loss = 0.25377136755417634
  Batch 1,900  of  46,094.    Elapsed: 0:30:30.
  current average loss = 0.2531838277570511
  Batch 1,920  of  46,094.    Elapsed: 0:30:49.
  current average loss = 0.2525870923651382
  Batch 1,940  of  46,094.    Elapsed: 0:31:08.
  current average loss = 0.25220467445162154
  Batch 1,960  of  46,094.    Elapsed: 0:31:27.
  current average loss = 0.251289528174972
  Batch 1,980  of  46,094.    Elapsed: 0:31:47.
  current average loss = 0.2508430253827211
  Batch 2,000  of  46,094.    Elapsed: 0:32:06.
  current average loss = 0.2502

  Batch 3,680  of  46,094.    Elapsed: 0:59:05.
  current average loss = 0.2270168463914635
  Batch 3,700  of  46,094.    Elapsed: 0:59:24.
  current average loss = 0.22674769656304775
  Batch 3,720  of  46,094.    Elapsed: 0:59:43.
  current average loss = 0.2267253182695738
  Batch 3,740  of  46,094.    Elapsed: 1:00:03.
  current average loss = 0.2264683445176
  Batch 3,760  of  46,094.    Elapsed: 1:00:22.
  current average loss = 0.22618925862405648
  Batch 3,780  of  46,094.    Elapsed: 1:00:41.
  current average loss = 0.22590773062627742
  Batch 3,800  of  46,094.    Elapsed: 1:01:01.
  current average loss = 0.2256517775383729
  Batch 3,820  of  46,094.    Elapsed: 1:01:20.
  current average loss = 0.2252854947729709
  Batch 3,840  of  46,094.    Elapsed: 1:01:39.
  current average loss = 0.2249426601617112
  Batch 3,860  of  46,094.    Elapsed: 1:01:59.
  current average loss = 0.22490120886902235
  Batch 3,880  of  46,094.    Elapsed: 1:02:18.
  current average loss = 0.2246

  Batch 5,460  of  46,094.    Elapsed: 1:27:41.
  current average loss = 0.21124300891772294
  Batch 5,480  of  46,094.    Elapsed: 1:28:01.
  current average loss = 0.2111916246667613
  Batch 5,500  of  46,094.    Elapsed: 1:28:20.
  current average loss = 0.21106194948066365
  Batch 5,520  of  46,094.    Elapsed: 1:28:39.
  current average loss = 0.21097005046904088
  Batch 5,540  of  46,094.    Elapsed: 1:28:58.
  current average loss = 0.21068946684942672
  Batch 5,560  of  46,094.    Elapsed: 1:29:18.
  current average loss = 0.2104968976798943
  Batch 5,580  of  46,094.    Elapsed: 1:29:37.
  current average loss = 0.21042979849989804
  Batch 5,600  of  46,094.    Elapsed: 1:29:56.
  current average loss = 0.21023890210781246
  Batch 5,620  of  46,094.    Elapsed: 1:30:15.
  current average loss = 0.21016474351936493
  Batch 5,640  of  46,094.    Elapsed: 1:30:35.
  current average loss = 0.21000484550200033
  Batch 5,660  of  46,094.    Elapsed: 1:30:54.
  current average loss =

  Batch 7,240  of  46,094.    Elapsed: 1:56:16.
  current average loss = 0.19915617323629764
  Batch 7,260  of  46,094.    Elapsed: 1:56:35.
  current average loss = 0.19891408840476563
  Batch 7,280  of  46,094.    Elapsed: 1:56:54.
  current average loss = 0.1988260669333113
  Batch 7,300  of  46,094.    Elapsed: 1:57:13.
  current average loss = 0.19869732335528792
  Batch 7,320  of  46,094.    Elapsed: 1:57:32.
  current average loss = 0.19860570534307997
  Batch 7,340  of  46,094.    Elapsed: 1:57:51.
  current average loss = 0.1983867161911062
  Batch 7,360  of  46,094.    Elapsed: 1:58:10.
  current average loss = 0.19827427410092913
  Batch 7,380  of  46,094.    Elapsed: 1:58:29.
  current average loss = 0.198091538454252
  Batch 7,400  of  46,094.    Elapsed: 1:58:48.
  current average loss = 0.19790927898705107
  Batch 7,420  of  46,094.    Elapsed: 1:59:07.
  current average loss = 0.19775833727458034
  Batch 7,440  of  46,094.    Elapsed: 1:59:27.
  current average loss = 0

...

  Batch 10,600  of  46,094.    Elapsed: 2:47:25.
  current average loss = 0.006804459296907886
  Batch 10,620  of  46,094.    Elapsed: 2:47:44.
  current average loss = 0.006795844807061943
  Batch 10,640  of  46,094.    Elapsed: 2:48:03.
  current average loss = 0.006814476431143428
  Batch 10,660  of  46,094.    Elapsed: 2:48:22.
  current average loss = 0.0068122181160970964
  Batch 10,680  of  46,094.    Elapsed: 2:48:41.
  current average loss = 0.0068091776870790295
  Batch 10,700  of  46,094.    Elapsed: 2:49:00.
  current average loss = 0.006797594160344216
  Batch 10,720  of  46,094.    Elapsed: 2:49:19.
  current average loss = 0.006816653857553033
  Batch 10,740  of  46,094.    Elapsed: 2:49:38.
  current average loss = 0.006835212436392039
  Batch 10,760  of  46,094.    Elapsed: 2:49:57.
  current average loss = 0.006824680330218774
  Batch 10,780  of  46,094.    Elapsed: 2:50:16.
  current average loss = 0.006816559400268583
  Batch 10,800  of  46,094.    Elapsed: 2:50:35.

  Batch 12,340  of  46,094.    Elapsed: 3:14:55.
  current average loss = 0.006962506143367471
  Batch 12,360  of  46,094.    Elapsed: 3:15:14.
  current average loss = 0.0069823003853794535
  Batch 12,380  of  46,094.    Elapsed: 3:15:33.
  current average loss = 0.006973580716236971
  Batch 12,400  of  46,094.    Elapsed: 3:15:52.
  current average loss = 0.006974422193881889
  Batch 12,420  of  46,094.    Elapsed: 3:16:12.
  current average loss = 0.006966641059775114
  Batch 12,440  of  46,094.    Elapsed: 3:16:31.
  current average loss = 0.0069609248832089585
  Batch 12,460  of  46,094.    Elapsed: 3:16:50.
  current average loss = 0.006970310328610436
  Batch 12,480  of  46,094.    Elapsed: 3:17:09.
  current average loss = 0.006961403199460137
  Batch 12,500  of  46,094.    Elapsed: 3:17:28.
  current average loss = 0.0069715786032227695
  Batch 12,520  of  46,094.    Elapsed: 3:17:46.
  current average loss = 0.006961362776413141
  Batch 12,540  of  46,094.    Elapsed: 3:18:05

  Batch 14,180  of  46,094.    Elapsed: 3:44:00.
  current average loss = 0.007118596516517001
  Batch 14,200  of  46,094.    Elapsed: 3:44:19.
  current average loss = 0.007142226933834107
  Batch 14,220  of  46,094.    Elapsed: 3:44:38.
  current average loss = 0.007153532649151251
  Batch 14,240  of  46,094.    Elapsed: 3:44:57.
  current average loss = 0.007147521799287508
  Batch 14,260  of  46,094.    Elapsed: 3:45:16.
  current average loss = 0.007144030218288644
  Batch 14,280  of  46,094.    Elapsed: 3:45:35.
  current average loss = 0.0071379134385638815
  Batch 14,300  of  46,094.    Elapsed: 3:45:54.
  current average loss = 0.0071542521876347825
  Batch 14,320  of  46,094.    Elapsed: 3:46:13.
  current average loss = 0.0071673196793555155
  Batch 14,340  of  46,094.    Elapsed: 3:46:31.
  current average loss = 0.007168695203913405
  Batch 14,360  of  46,094.    Elapsed: 3:46:50.
  current average loss = 0.007166575978190123
  Batch 14,380  of  46,094.    Elapsed: 3:47:09

  Batch 15,920  of  46,094.    Elapsed: 4:11:31.
  current average loss = 0.007200527547345457
  Batch 15,940  of  46,094.    Elapsed: 4:11:50.
  current average loss = 0.007206828131255715
  Batch 15,960  of  46,094.    Elapsed: 4:12:09.
  current average loss = 0.00719904228357143
  Batch 15,980  of  46,094.    Elapsed: 4:12:28.
  current average loss = 0.007194003772548975
  Batch 16,000  of  46,094.    Elapsed: 4:12:47.
  current average loss = 0.0071877174839024465
  Batch 16,020  of  46,094.    Elapsed: 4:13:06.
  current average loss = 0.007180270291199493
  Batch 16,040  of  46,094.    Elapsed: 4:13:25.
  current average loss = 0.0071771462704175275
  Batch 16,060  of  46,094.    Elapsed: 4:13:44.
  current average loss = 0.007172221258659861
  Batch 16,080  of  46,094.    Elapsed: 4:14:03.
  current average loss = 0.0071736584772228
  Batch 16,100  of  46,094.    Elapsed: 4:14:22.
  current average loss = 0.0071783917479217715
  Batch 16,120  of  46,094.    Elapsed: 4:14:41.
 

KeyboardInterrupt: 